# Homework Assignment 4: Feature Selection
As in the previous assignments, in this homework assignment you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This assignment will utilize a copy of the feature dataset we have been working with, but has had some additional preprocessing like clipping, z-score normalization, and range normalization performed on it. 

## Step 1: Downloading the Data

This assignment will continue to only use [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB). Recall that in Homework 1, we started to construct the analytics base table for our [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM). In that assignment, we read the data from the two subdirectories, __FL__ and __NF__, of the __partition1__ direcotry. These two subdirectories represented the two classes of our target feature in the solar flare prediction problem we will be attempting to solve this semester. We then processed these samples of multivariate time series to construct descriptive features for each sample, and then placed them into our analytics base table.

Then, in Homework 2, you utilized a set of extracted descriptive features much like what you were asked to construct in Homework 1. However, this dataset contained many more extracted features than you were asked to compute for Homework 1 (>800), so we needed to explore the data to find data quality issues and identify ways to address these issues. Below are the links to the full extracted feature for all of partition 1, and a toy representative dataset, that were used as input to Homework 2.

- [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv)
- [Toy Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_partition1ExtractedFeatures.csv)

Then, in Homework 3, you were asked to perform additional data preprocessing on data that would have been produced from Homework 2. The links to those files are below.  

- [Full Cleaned Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/cleaned_partition1ExtractedFeatures.csv)
- [Toy Cleaned Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_cleaned_partition1ExtractedFeatures.csv)
- [Data Quality Table for Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/data_quality_table.csv)

The preprocessing you were asked to do in Homework 3 included finding features with large ranges and features with a large number of outliers. You were asked to clip some of the outliers for the features you found and were also asked to perform a few different types of scaling, such as decimal and z-score.  I have done much more of this preprocessing for you, and have constructed a set of files for you to use for this assignment.  

Below you will find the full normalized and a toy normalized data file.  Some of the questions I've asked of you this week do take some time for the processing to complete. So, if you find yourself running out of time for them to complete processing, I suggest you switch to using the toy data. Your results don't need to be based on the full data (though that is preferable), your code just needs to do what is asked regardless of the file used as input.

- [Full Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition1ExtractedFeatures.csv)
- [Toy Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition1ExtractedFeatures.csv)

Now that you have the normalized features csv files, you will load that data into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method. 

In [ ]:
%matplotlib inline
import os
import pandas as pd
from pandas import DataFrame 
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LassoLarsCV

In [ ]:
data_dir = '/data/FDS'
data_file = 'normalized_partition1ExtractedFeatures.csv'

In [ ]:
abt = pd.read_csv(os.path.join(data_dir, data_file))

### Q1 (20 points)

The provided data has a NaN or Inf in virtually every feature, but the feature selection methods that we will utilize require us to remove these values from our data.  So, this question you should process the normalized partition one data and remove the features that have greater than 1% of the observations with either the NaN or Inf values. For the remaining feeatures, you should assign the median value to the samples that have either Nan or Inf.  These operations should be performed in place on the input DataFrame for the function below.

In [ ]:
def clean_nan_and_inf(data:DataFrame):
    data.replace([np.inf, -np.inf], np.nan, inplace=True) 
    data.dropna(thresh = len(data) * .99, axis = 1, inplace = True)
    for i in range(len(data)):
        data.iloc[:,i].fillna(data.iloc[:,i].median(), inplace=True) 

In [ ]:
abt_cpy = abt.copy()
clean_nan_and_inf(abt_cpy)

### Q2 (20 points)

Using the cleaned data from Q1, you now need to perform feature selection on the dataset and take the 5 most useful features for classification. For now, we will utilize all the different labels in our evaluation of features (i.e. NF, B, C, M, X).  To perform the ranking you will utilize the ANOVA F-Value to select the top 5 features.

Use the [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) class from [scikit-learn Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection). The function of [scikit-learn f_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif) should be used to initialize the class.  

Once you have used the SelectKBest class to perform feature selection, you shuld construct a new DataFrame with the `lab` class labels, and the selected 5 features. In the next quesiton, you will plot them to compare, just like we did in the last assignment. **Note:** Using the fit_transform function of the SelectKBest class gives you the slected features, but it does not give the names. So, you should utilzie another function to get the names of the features and then manually construct a DataFrame yourself. 

In [ ]:
numFeat = 5
y = abt_cpy["lab"]
x = abt_cpy.drop('lab', axis=1)
f_df = pd.DataFrame(SelectKBest(f_classif, k=numFeat).fit_transform(x, y))
sekectbest_classif_df = pd.concat([y, f_df], axis=1)

### Q3 (5 points)

For this question plot the features from Q2 using a Scatter Plot Matrix (pair plot) of these features. To accomplish this, use the seaborn [pairplot](https://seaborn.pydata.org/generated/seaborn.pairplot.html) function.

If your feature selection and cleaning above works correctly, you should see that several features were chosen that: 

* Look to be highly correlated with one another.

* Show most if not all of the X class flares as a distinct group apart from all of the other classes. 

This shows us that there are features that can be used to group many of the X class flares into a pure cluster, and also that many of these features are redundant.  So, maybe we need to look into selecting features using another feature selection method.  

In [ ]:
sns.pairplot(sekectbest_classif_df, hue='lab', diag_kind='hist')

### Q4 (20 points)

Just like in Q2 start by using the cleaned data from Q1, you will perform feature selection on the dataset and take the 5 most useful features for classification. We are still utilizing all the different labels in our evaluation of features (i.e. NF, B, C, M, X).  For this question, instead of using the ANOVA F-Value to select the top 5 features, you will be utilizing mutual information to select the features.

Use the [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) class from [scikit-learn Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection). The function of [scikit-learn mutual_info_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif) should be used to initialize the class.  

Once you have used the SelectKBest class to perform feature selection, you shuld construct a new DataFrame with the `lab` class labels, and the selected 5 features. In the next quesiton, you will plot them to compare, just like we did in Q3. 

In [ ]:
numFeat = 5
y = abt_cpy["lab"]
x = abt_cpy.drop('lab', axis=1)
mutual_data = pd.DataFrame(SelectKBest(mutual_info_classif, k=numFeat).fit_transform(x, y))
sekectbest_mutual_df = pd.concat([y, mutual_data], axis=1)

### Q5 (5 points)

For this question plot the features from Q4 using a Scatter Plot Matrix (pair plot) of these features. To accomplish this, use the seaborn [pairplot](https://seaborn.pydata.org/generated/seaborn.pairplot.html) function.

In this plot, you should see that several of the features selected by the the mutual information method work well to cluster the NF samples, but don't do quite as good of a job at clustering the X and M class samples. 

Like the features we found using ANOVA F-Value, and that we plotted in the last assignment, we still don't see any two as being able to correctly distinguish between all the different classes in our dataset.

In [ ]:
sns.pairplot(sekectbest_mutual_df, hue='lab', diag_kind='hist')

### Q6 (20 points)

Just like in the previous two feature selection questions, start by using the cleaned data from Q1. Again, you will perform feature selection on the dataset and take the 5 most useful features for classification. We will continue utilize all the different labels in our evaluation of features (i.e. NF, B, C, M, X).  For this question, instead of using the ANOVA F-Value or mutual information to select the features, you will utilize the chi squared feature selection.

Use the [SequentialFeatureSelector](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html#sklearn.feature_selection.SequentialFeatureSelector) class from [scikit-learn Sequential Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#sequential-feature-selection). The function of [scikit-learn LassoLarsCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsCV.html#sklearn.linear_model.LassoLarsCV) should be used to initialize the class. There is a [discussion page](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_select_from_model_diabetes.html#sphx-glr-auto-examples-feature-selection-plot-select-from-model-diabetes-py) that has an example of utilizing the LassoCV for the SequentialFeatureSelector. Use that and perform feature selection in the forward direction, which LassoLarsCV can be a replacement for. 

For this feature selection to work, you will need to convert the categorical string labels to numeric values, you may want to use the [map](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html) function to change the values of the column using a dictionary. You could use 0 for NF, 1 for B, etc.

Once you have used the SequentialFeatureSelector class to perform feature selection, you shuld construct a new DataFrame with the original `lab` class labels, and the selected 5 features. In the next quesiton, you will plot them to compare, just like we did in Q5. 

In [ ]:
y = abt_cpy['lab'].replace({'NF':0,'C':1,'B':2,'M':3,'X':4})
x = abt_cpy.drop('lab', axis=1)
LassoLarsCV_df = pd.DataFrame(SequentialFeatureSelector(estimator=LassoLarsCV(), n_features_to_select=numFeat, direction='forward').fit_transform(x,y))
sekectbest_LassoLarsCV_df = pd.concat([y, LassoLarsCV_df], axis=1)

### Q7 (5 points)

For this question plot the features from Q6 using a Scatter Plot Matrix (pair plot) of these features. To accomplish this, use the seaborn [pairplot](https://seaborn.pydata.org/generated/seaborn.pairplot.html) function.

In this plot, you should see that several of the features selected by the the sequential feature selector have a few different clusters of M and X, as well as others. It seems we are starting to find some good feature combinations, thogh still not perfect.


In [ ]:
sns.pairplot(sekectbest_LassoLarsCV_df, hue='lab', diag_kind='hist')

### Q8 (5 points)

Now that we have used several different methods to select features from our dataset, we need to join them and then save them to disk for later use.  So, for this question you will construct a dataframe that contains 

- The label `lab` column
- The 5 features found in Q2
- The 5 features found in Q4
- The 5 features found in Q6 

If there are duplicates caused by more than one selection method finding the same feature, then only include one copy in the final DataFrame. **Note:** If you look into the [DataFrame.columns](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.columns.html) parameter, you will see that it is an [Index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.html#pandas.Index) type, and this object type has a nice function called [difference](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.difference.html) that can be easily used to find which DataFrame columns are not already part of another DataFrame.

In [ ]:
df1 = pd.concat([f_df, mutual_data, LassoLarsCV_df], axis=1).drop_duplicates(keep=False)
df1 = df1.drop('lab', axis=1)
new_df = pd.concat(abt_cpy['lab'], df1, axis=1)
new_df.to_csv(os.path.join(data_dir, 'selected_features.csv'))

## End

Please use the `Kernel` menu to `Restart & Clear Output` prior to saving and submitting your assignment.